In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkSession.builder.appName("pysparkDataframes").getOrCreate()

In [3]:
dfStorm2019 = sc.read.format('csv')\
                .option('header', 'true')\
                .option('delimiter', ',')\
                .option('inferSchema', 'true')\
                .load('../pyspark/data/StormEvents_details-ftp_v1.0_d2019_c20200317.csv.gz')

In [4]:
dfFatalities2019 = sc.read.format('csv')\
                    .option('header', 'true')\
                    .option('delimiter', ',')\
                    .option('inferSchema', 'true')\
                    .load('../pyspark/data/StormEvents_fatalities-ftp_v1.0_d2019_c20200317.csv.gz')

In [5]:
dfLocations2019 = sc.read.format('csv')\
                    .option('header', 'true')\
                    .option('delimiter', ',')\
                    .option('inferSchema', 'true')\
                    .load('../pyspark/data/StormEvents_locations-ftp_v1.0_d2019_c20200317.csv.gz')

In [6]:
df_storms = dfStorm2019.toPandas()

In [7]:
df_storms.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201905,9,1554,201905,9,1830,137295,824116,TEXAS,48,...,7.0,NNE,SAN GERONIMO,29.7898,-98.6406,29.7158,-98.7744,Thunderstorms developed along a cold front as ...,Thunderstorms produced heavy rain that led to ...,CSV
1,201907,15,1640,201907,15,1641,140217,843354,MINNESOTA,27,...,5.0,E,ROCK CREEK,45.7700,-92.8700,45.7700,-92.8700,An area of low pressure over southern Manitoba...,A 14-inch tree fell due to the winds and damag...,CSV
2,201910,20,2223,201910,20,2223,142648,861581,TEXAS,48,...,1.0,N,EDGEWOOD,32.7100,-95.8800,32.7100,-95.8800,Thunderstorms erupted across the DFW Metroplex...,A trained spotter measured a wind gust of 67 M...,CSV
3,201910,20,2312,201910,20,2312,142648,861584,TEXAS,48,...,5.0,WSW,AZLE,32.8700,-97.6100,32.8700,-97.6100,Thunderstorms erupted across the DFW Metroplex...,The local police department reported a wind gu...,CSV
4,201910,20,2236,201910,20,2236,142648,861582,TEXAS,48,...,0.0,N,MINERAL WELLS,32.8000,-98.1000,32.8000,-98.1000,Thunderstorms erupted across the DFW Metroplex...,A wind gust of 75 MPH was measured by the Auto...,CSV


In [8]:
df_storms.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [9]:
df_storms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67337 entries, 0 to 67336
Data columns (total 51 columns):
BEGIN_YEARMONTH       67337 non-null int32
BEGIN_DAY             67337 non-null int32
BEGIN_TIME            67337 non-null int32
END_YEARMONTH         67337 non-null int32
END_DAY               67337 non-null int32
END_TIME              67337 non-null int32
EPISODE_ID            67337 non-null int32
EVENT_ID              67337 non-null int32
STATE                 67337 non-null object
STATE_FIPS            67337 non-null int32
YEAR                  67337 non-null int32
MONTH_NAME            67337 non-null object
EVENT_TYPE            67337 non-null object
CZ_TYPE               67337 non-null object
CZ_FIPS               67337 non-null int32
CZ_NAME               67337 non-null object
WFO                   67337 non-null object
BEGIN_DATE_TIME       67337 non-null object
CZ_TIMEZONE           67337 non-null object
END_DATE_TIME         67337 non-null object
INJURIES_DIRECT       

Delete the columns with less information:

### 1. Predict Event type

In [10]:
df_storms.loc[:, ['EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'EVENT_TYPE']].head(10)

,EPISODE_NARRATIVE,EVENT_NARRATIVE,EVENT_TYPE
0,Thunderstorms developed along a cold front as ...,Thunderstorms produced heavy rain that led to ...,Flash Flood
1,An area of low pressure over southern Manitoba...,A 14-inch tree fell due to the winds and damag...,Thunderstorm Wind
2,Thunderstorms erupted across the DFW Metroplex...,A trained spotter measured a wind gust of 67 M...,Thunderstorm Wind
3,Thunderstorms erupted across the DFW Metroplex...,The local police department reported a wind gu...,Thunderstorm Wind
4,Thunderstorms erupted across the DFW Metroplex...,A wind gust of 75 MPH was measured by the Auto...,Thunderstorm Wind
5,Thunderstorms erupted across the DFW Metroplex...,"Tornadic wind damage was observed in Rockwall,...",Tornado
6,A strong mid-level disturbance and cold front ...,Quarter size hail reported at Okemo Ski resort.,Hail
7,A strong mid-level disturbance and cold front ...,A few trees and power lines downed by thunders...,Thunderstorm Wind
8,Fast moving showers came onshore across the Br...,Weatherflow site XCOO at the Cocoa Beach Pier ...,Marine Thunderstorm Wind
9,Fast moving showers came onshore across the Br...,US Air Force wind tower 3131 measured a peak w...,Marine Thunderstorm Wind
